In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
MINUTO_ANALISADO = 7

In [2]:
df = pd.read_csv('../../../data/crawler/unified-events-time-statistics.csv')
filtro = df['time'] == MINUTO_ANALISADO
df = df[filtro]
df

,golId,time,result,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,...,RED:third_tower_top,RED:third_tower_mid,RED:third_tower_bot,RED:inhibitor_top,RED:inhibitor_mid,RED:inhibitor_bot,RED:baron,RED:elder_dragon,RED:nexus_tower,RED:nexus
7,35797,7,0,0,0.00,0.00,0,0.00,0.00,0,...,0,0,0,0,0,0,0,0,0,0
48,35798,7,0,0,0.00,0.00,2,0.00,1.18,0,...,0,0,0,0,0,0,0,0,0,0
97,35799,7,1,2,0.00,1.00,5,1.00,5.78,0,...,0,0,0,0,0,0,0,0,0,0
136,35800,7,1,3,0.33,4.25,0,0.00,0.00,0,...,0,0,0,0,0,0,0,0,0,0
171,35801,7,0,0,0.00,0.00,3,0.33,2.08,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195347,49142,7,0,2,0.50,3.67,8,0.62,5.18,0,...,0,0,0,0,0,0,0,0,0,0
195376,53243,7,0,4,0.25,2.83,10,0.80,8.38,2,...,0,0,0,0,0,0,0,0,0,0
195407,53244,7,1,5,0.80,2.91,3,1.00,3.75,10,...,0,0,0,0,0,0,0,0,0,0
195440,45053,7,0,0,0.00,0.00,2,0.00,4.71,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','time'],axis=1)
X.head()

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,RED:third_tower_top,RED:third_tower_mid,RED:third_tower_bot,RED:inhibitor_top,RED:inhibitor_mid,RED:inhibitor_bot,RED:baron,RED:elder_dragon,RED:nexus_tower,RED:nexus
7,0,0.00,0.00,0,0.00,0.00,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0.00,0.00,2,0.00,1.18,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
97,2,0.00,1.00,5,1.00,5.78,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
136,3,0.33,4.25,0,0.00,0.00,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
171,0,0.00,0.00,3,0.33,2.08,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y,random_state=42)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test,scaler

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

executions = pd.DataFrame()

for i in range(50):
  print('execução número ' + str(i))
  X_train,X_test,y_train,y_test,scaler = preprocess_input(X,y)

  models = {
      'Logistic Regression': LogisticRegression(max_iter=10000),
      'Support Vector Machine (RBF Kernel)': SVC(C=1,gamma=0.01,kernel='sigmoid',max_iter=50000,probability=True),
      'Decission Tree': DecisionTreeClassifier(),
      'Adaboost': AdaBoostClassifier(),
      'Random Forest': RandomForestClassifier(),
      'Gradient Boosting Classifier': GradientBoostingClassifier(),
      'KNN': KNeighborsClassifier(n_neighbors=3),
      'Gaussian NB': GaussianNB()
  }

  for name, model in models.items():
    model.fit(X_train,y_train)

  scores_list = []

  for name,model in models.items():    
      scores_list.append({
      'model': name,
      'game_minutes': MINUTO_ANALISADO,
      'execution': i,
      'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
      'Precision':  precision_score(y_test,model.predict(X_test)),
      'Recall': recall_score(y_test,model.predict(X_test)),
      'F1-Score': f1_score(y_test,model.predict(X_test)),
      'auc': roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
      })
  # scores = pd.DataFrame(scores_list)
  executions = executions.append(scores_list)

execução número 0
execução número 1
execução número 2
execução número 3
execução número 4
execução número 5
execução número 6
execução número 7
execução número 8
execução número 9
execução número 10
execução número 11
execução número 12
execução número 13
execução número 14
execução número 15
execução número 16
execução número 17
execução número 18
execução número 19
execução número 20
execução número 21
execução número 22
execução número 23
execução número 24
execução número 25
execução número 26
execução número 27
execução número 28
execução número 29
execução número 30
execução número 31
execução número 32
execução número 33
execução número 34
execução número 35
execução número 36
execução número 37
execução número 38
execução número 39
execução número 40
execução número 41
execução número 42
execução número 43
execução número 44
execução número 45
execução número 46
execução número 47
execução número 48
execução número 49


In [6]:
avg_scores_list = []

for name, model in models.items():

    avg_balanced_acc = 0

    model_metrics = executions.loc[executions['model'] == name]
    avg_balanced_acc = model_metrics['Balanced Accuracy'].sum() / len(model_metrics['Balanced Accuracy'])
    avg_precision = model_metrics['Precision'].sum() / len(model_metrics['Precision'])
    avg_recall = model_metrics['Recall'].sum() / len(model_metrics['Recall'])
    avg_f_score = model_metrics['F1-Score'].sum() / len(model_metrics['F1-Score'])
    avg_auc = model_metrics['auc'].sum() / len(model_metrics['auc'])

    avg_scores_list.append({
      'model': name,
      'model_obj': model,
      'game_minutes': MINUTO_ANALISADO,
      'Balanced Accuracy': avg_balanced_acc,
      'Precision': avg_precision,
      'Recall': avg_recall,
      'F1-Score': avg_f_score,
      'auc': avg_auc
      })
avg_scores = pd.DataFrame(avg_scores_list)

In [7]:
ordered_scores = avg_scores.sort_values(by='auc', ascending=False)

In [8]:
ordered_scores

,model,model_obj,game_minutes,Balanced Accuracy,Precision,Recall,F1-Score,auc
5,Gradient Boosting Classifier,([DecisionTreeRegressor(criterion='friedman_ms...,7,0.638571,0.641585,0.707177,0.672786,0.690407
3,Adaboost,"(DecisionTreeClassifier(max_depth=1, random_st...",7,0.630158,0.636632,0.687400,0.661043,0.683110
0,Logistic Regression,LogisticRegression(max_iter=10000),7,0.626333,0.630970,0.695375,0.661608,0.678878
1,Support Vector Machine (RBF Kernel),"SVC(C=1, gamma=0.01, kernel='sigmoid', max_ite...",7,0.618733,0.623919,0.690590,0.655564,0.675232
4,Random Forest,"(DecisionTreeClassifier(max_features='auto', r...",7,0.625226,0.634870,0.669793,0.651828,0.674985
7,Gaussian NB,GaussianNB(),7,0.501009,0.521703,0.996810,0.684932,0.666099
6,KNN,KNeighborsClassifier(n_neighbors=3),7,0.565304,0.583732,0.583732,0.583732,0.584932
2,Decission Tree,DecisionTreeClassifier(),7,0.550924,0.571079,0.558341,0.564618,0.547194


In [9]:
# Ensemble Voting Classifier
from sklearn.ensemble import VotingClassifier

# Criando o ensemble com os classificadores
ensemble_classifier = VotingClassifier(estimators=[
    (ordered_scores.iloc[0]['model'], ordered_scores.iloc[0]['model_obj']),
    (ordered_scores.iloc[1]['model'], ordered_scores.iloc[1]['model_obj']),
    (ordered_scores.iloc[2]['model'], ordered_scores.iloc[2]['model_obj'])
], voting='soft')

ensemble_classifier = ensemble_classifier.fit(X_train, y_train)

In [10]:
ensemble_scores_list = []
ensemble_scores_list.append({
    'model': 'Ensemble Classifier',
    'model_obj': ensemble_classifier,
    'game_minutes': MINUTO_ANALISADO,
    'Balanced Accuracy': balanced_accuracy_score(y_test,ensemble_classifier.predict(X_test)),
    'Precision':  precision_score(y_test,ensemble_classifier.predict(X_test)),
    'Recall': recall_score(y_test,ensemble_classifier.predict(X_test)),
    'F1-Score': f1_score(y_test,ensemble_classifier.predict(X_test)),
    'auc': roc_auc_score(y_test,ensemble_classifier.predict_proba(X_test)[:,1])
})

In [11]:
ensemble_scores = pd.DataFrame(ensemble_scores_list)
ensemble_scores_list

[{'model': 'Ensemble Classifier',
  'model_obj': VotingClassifier(estimators=[('Gradient Boosting Classifier',
                                GradientBoostingClassifier()),
                               ('Adaboost', AdaBoostClassifier()),
                               ('Logistic Regression',
                                LogisticRegression(max_iter=10000))],
                   voting='soft'),
  'game_minutes': 7,
  'Balanced Accuracy': 0.6405253189792663,
  'Precision': 0.6434782608695652,
  'Recall': 0.7081339712918661,
  'F1-Score': 0.6742596810933941,
  'auc': 0.6889481437178805}]

In [12]:
ensemble_scores

,model,model_obj,game_minutes,Balanced Accuracy,Precision,Recall,F1-Score,auc
0,Ensemble Classifier,VotingClassifier(estimators=[('Gradient Boosti...,7,0.640525,0.643478,0.708134,0.67426,0.688948


In [13]:
models = pd.concat([ensemble_scores,avg_scores])
ordered_models = models.sort_values(by='auc', ascending=False)
best_model = ordered_models.iloc[0]['model_obj']
best_model

GradientBoostingClassifier()

In [14]:
header = ['model', 'game_minutes', 'auc']
models.to_csv('../../../data/models-analysis/models-time-metric.csv', columns = header, mode='a',index=False,header=False)